# U-Net

<img src="images/unet.png" width="30%" height="10%">


Bu notebook, görüntü segmentasyonu için U-Net modelinin bir PyTorch uygulamasıdır.Bu Model, Olaf Ronneberger, Philipp Fischer ve Thomas Brox tarafından yazılan [U-Net: Biyomedikal Görüntü Segmentasyonu için Evrişimli Ağlar] (https://arxiv.org/abs/1505.04597) makalesine dayanmaktadır.

 ### 1. Kütüphaneleri Içeri Aktarma

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import CenterCrop 

### 2. U-Net Modelinin Bloklarını Tanımlama

<img src="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png" width="70%" height="70%">


Yukarıdan da anlaşılabileceği üzere, modelin içinde çeşitli bölümler tekrar etmektedir. Örneğin alt örnekleme (Down Sampling) ve üst örnekleme (Up Sampling) işlemleri her biri 4 kez tekrar edilmektedir. Bu yüzden modeli iki parçaya ayıracağız: **Alt Örnekleme Bloğu** ve **Üst Örnekleme Bloğu**. Daha sonra bu iki bloğu birleştirerek U-Net modelini oluşturacağız.

##### &nbsp; **2.1.** Alt Örnekleme Bloğu (Down Sampling Block - Encoder)
##### &nbsp; **2.2.** Üst Örnekleme Bloğu (Up Sampling Block - Decoder)
##### &nbsp; **2.3.** U-Net Modeli

### 2.1. Alt Örnekleme Bloğu (Down Sampling Block - Encoder)

![Down Sampling Block](images/down_sampling_block.png)

Bu blok, her biri 3x3 kernel boyutuna sahip iki evrişim katmanı ve 2x2 kernel boyutu ve 2x2 stride değerine sahip bir max pooling katmanı içerir. Her evrişim katmanının çıktısı ReLU aktivasyon fonksiyonundan geçirilir ve Batch Normalization uygulanır. Max pooling katmanı, girdinin mekansal boyutlarını (Spatial Dimension) yarıya düşürmek için kullanılır. Evrişim katmanlarının filtre sayısı, her alt örnekleme bloğu sonrasında iki kat artırılır.

In [2]:
class DownSamplingBlock(nn.Module):
    def __init__(self, in_filter_num, out_filter_num=None):
        super().__init__()
        
        if out_filter_num is None:
            out_filter_num = in_filter_num*2

        self.batch_norm1 = nn.BatchNorm2d(num_features=out_filter_num)
        self.batch_norm2 = nn.BatchNorm2d(num_features=out_filter_num)
        
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels=in_filter_num, out_channels=out_filter_num, kernel_size=(3,3))
        self.conv2 = nn.Conv2d(in_channels=out_filter_num, out_channels=out_filter_num, kernel_size=(3,3))
        self.max_pool = nn.MaxPool2d(kernel_size=(2,2), stride=2)

    def forward(self, x):
        # x : (batch_size, in_filter_num, dim, dim)
        x = self.relu(self.conv1(x)) # x : (batch_size, out_filter_num, dim-2, dim-2)
        x = self.batch_norm1(x) # x : (batch_size, out_filter_num, dim-2, dim-2)
        x = self.relu(self.conv2(x)) # x : (batch_size, out_filter_num, dim-4, dim-4)
        x = self.batch_norm2(x) # x : (batch_size, out_filter_num, dim-4, dim-4)
        out = self.max_pool(x) # x : (batch_size, out_filter_num, (dim-4)/2 , (dim-4)/2 )
        return out, x        

### 2.2. Üst Örnekleme Bloğu (Up Sampling Block - Decoder)

![Up Sampling Block](images/up_sampling_block.png)

Üst örnekleme bloğu, önce Up Convolution (Yukarı Evrişim), ardından birleştirme (concatenating), 2 evrişim katmanı ve her evrişim katmanından sonra ReLU ve Batch Normalizasyon işlemleri içerir. Bu bloğun amacı, alt-örnekleme bloğunda yapılan boyut indirgemeyi tersine çevirerek, girdi boyutlarını geri kazanmaktır.

In [3]:
class UpSamplingBlock(nn.Module):
    def __init__(self, in_filter_num, out_filter_num=None):
        super().__init__()

        if out_filter_num is None:
            out_filter_num = in_filter_num//2

        self.batch_norm1 = nn.BatchNorm2d(num_features=out_filter_num)
        self.batch_norm2 = nn.BatchNorm2d(num_features=out_filter_num)

        self.relu = nn.ReLU()
        self.up_conv = nn.ConvTranspose2d(in_channels=in_filter_num,out_channels=out_filter_num, kernel_size=(2,2), stride=2)
        self.conv1 = nn.Conv2d(in_channels=in_filter_num,out_channels=out_filter_num, kernel_size=(3,3))
        self.conv2 = nn.Conv2d(in_channels=out_filter_num,out_channels=out_filter_num, kernel_size=(3,3)) 

    def forward(self, x, enc_out):

        x = self.up_conv(x)
        
        x_size = tuple(x.shape[-2:])
        crop = CenterCrop(size=x_size)
        cropped_enc = crop(enc_out)
        concatenated_input = torch.cat((cropped_enc,x),dim=1)
    
        x = self.relu(self.conv1(concatenated_input))
        x = self.batch_norm1(x)        
        x = self.relu(self.conv2(x))
        out = self.batch_norm2(x)
        return out
         

### 2.3. U-Net

<img src="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png" centered width="60%" height="60%">

Bu son blokta U-Net modelini tamamlayacağız. İlk olarak, 4 Alt Örnekleme Bloğu (Down Sampling Block) ve 4 Üst Örnekleme Bloğu (Up Sampling Block) tanımlayacağız. Aralarında bir son çift evrişim katmanı olmalıdır. Son olarak, son bloğu tanımlayacağız ki bu, 1x1 kernel boyutuna sahip bir evrişim katmanı olacak. Bu katman, nihai tahmini elde etmek için kullanılacak.

In [4]:
class UNet(nn.Module):
    def __init__(self, in_channel_num, out_channel_num, filter_num=64):
        super().__init__()
        
        self.relu = nn.ReLU()

        self.downsample_1 = DownSamplingBlock(in_filter_num=in_channel_num, out_filter_num=filter_num) # in_filter_num= 3, out_filter_num= 64
        self.downsample_2 = DownSamplingBlock(in_filter_num=filter_num) # in_filter_num= 64, out_filter_num= 128
        self.downsample_3 = DownSamplingBlock(in_filter_num=filter_num*2) # in_filter_num= 128, out_filter_num= 256
        self.downsample_4 = DownSamplingBlock(in_filter_num=filter_num*4) # in_filter_num= 256, out_filter_num= 512

        self.bottom_conv1 = nn.Conv2d(in_channels=filter_num*8, out_channels= filter_num*16, kernel_size=(3,3)) # in_filter_num= 512, out_filter_num= 1024
        self.bottom_conv2 = nn.Conv2d(in_channels=filter_num*16, out_channels= filter_num*16, kernel_size=(3,3)) # in_filter_num= 1024, out_filter_num= 1024

        self.upsample_1  = UpSamplingBlock(in_filter_num=filter_num*16) # in_filter_num= 1024, out_filter_num= 512
        self.upsample_2  = UpSamplingBlock(in_filter_num=filter_num*8) # in_filter_num= 512, out_filter_num= 256
        self.upsample_3  = UpSamplingBlock(in_filter_num=filter_num*4) # in_filter_num= 256, out_filter_num= 128
        self.upsample_4  = UpSamplingBlock(in_filter_num=filter_num*2) # in_filter_num= 128, out_filter_num= 64

        self.top_conv = nn.Conv2d(in_channels=filter_num, out_channels=out_channel_num, kernel_size=(1,1))

        # In total 23 conv layers

    def forward(self, x):
        # x : (batch_size, 1, 572, 572)
        x, enc_out1 = self.downsample_1(x) # x : (batch_size, 64 , 284, 284), enc_out1: (batch_size, 64 , 568, 568)
        x, enc_out2 = self.downsample_2(x) # x : (batch_size, 128, 140, 140), enc_out2: (batch_size, 128, 280, 280)
        x, enc_out3 = self.downsample_3(x) # x : (batch_size, 256, 68 , 68 ), enc_out3: (batch_size, 256, 136, 136)
        x, enc_out4 = self.downsample_4(x) # x : (batch_size, 512, 32 , 32 ), enc_out4: (batch_size, 512, 64 , 64 )

        x = self.relu(self.bottom_conv1(x)) # x : (batch_size, 1024 , 30, 30)
        x = self.relu(self.bottom_conv2(x)) # x : (batch_size, 1024 , 28, 28)

        x = self.upsample_1(x, enc_out4) # x : (batch_size, 512 , 52, 52)
        x = self.upsample_2(x, enc_out3) # x : (batch_size, 256 , 100, 100)
        x = self.upsample_3(x, enc_out2) # x : (batch_size, 128 , 196, 196)
        x = self.upsample_4(x, enc_out1) # x : (batch_size, 64 , 388, 388)
        out = self.top_conv(x) # out : (batch_size, 2 , 388, 388)

        return out


## Son
### Eğer herhangi bir sorunuz olursa, lütfen çekinmeden iletişime geçiniz.

Mail: i_konak@hotmail.com
<br>
Linkedin: https://www.linkedin.com/in/ismail-konak/
<br>
GitHub: https://github.com/IsmailKonak